# PSYC 193: Perception and Computation 
## Lab 4: kNN and linear classification

In this lab, we will continue working with an image dataset used in a recent computer vision paper by [Sangkloy et al.](https://dl.acm.org/doi/abs/10.1145/2897824.2925954). 

**Learning objectives**
* k-nearest-neighbor classification
* support vector machine classification
* logistic regression classification

**Submission instructions**
1. Please rename the notebook by replacing `YOURUSERNAME` in the filename with your actual UCSD AD username. 
2. Before submitting your assignment, sure that your notebook can run from "top to bottom," executing the code in every code cell without returning fatal errors. An easy way to verify this is to click "Kernel" above in the tool bar, and try selecting "Restart & Run All."
3. Once you have verified that your notebook can run "top to bottom" without issues, click "File" in the toolbar above, then "Download as," then "PDF via LaTeX" to download a PDF version of your notebook. 
4. Upload this PDF version of your notebook to Canvas before 5pm the next class period. 

### setup

In [ ]:
## load generally useful python modules
import os
import numpy as np
import pandas as pd
from PIL import Image
import requests
from io import BytesIO
import seaborn as sns

import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize, getfigs
%matplotlib inline
from IPython.display import clear_output

### load in dataset

In [ ]:
## import image metadata (from Sangkloy et al. (2016))
from photodraw32_metadata import metadata
M = pd.DataFrame(metadata)

### retrieve images and features from lab3

In [ ]:
## fill in name of directory that contains images
prefix_dir = '../lab3'
data_dir = os.path.join(prefix_dir,'images')
feature_dir = os.path.join(prefix_dir,'features')

#### extract VGG features

In [ ]:
## helper function to perform channel normalization
def normalize(X):
    X = X - X.mean(0)
    X = X / np.maximum(X.std(0), 1e-5)
    return X

## pre-extracted, only extract if you wish to overwrite
extract=False
if extract:
    cmd_string = "python extract_features.py --data={} --out_dir={}".format(data_dir,feature_dir)
    os.system(cmd_string)
    
## load in feature matrix and apply preprocessing (channel-wise normalization)
VF = normalize(np.load(os.path.join(feature_dir,'FEATURES_FC6_IMAGES.npy')))
num_vgg_feats = VF.shape[1]

## load in metadata corresponding to VGG features
VM = pd.read_csv(os.path.join(feature_dir,'METADATA_images.csv'))    

## join feature matrix and metadata to form single VGG dataframe
V = VM.join(pd.DataFrame(VF))

## create new columns to make it easier to sort into categories alphabetically
V['category'] = V.apply(lambda x: x['image_id'].split('_')[-2], axis=1)
V.loc[V['category']=='(sedan)', 'category'] = 'car_(sedan)' ## deal with exception, so categories sort properly

V['img_ind'] = VM.apply(lambda x: x['image_id'].split('_')[-1], axis=1)

## sort rows by category, then by img_ind
V.sort_values(by=['category','img_ind'], ascending=True, inplace=True)

### k-nearest-neighbor (kNN) classification
Recommended reading: https://cs231n.github.io/classification/ (markdown in this lab was adapted from these course notes)

The _Nearest Neighbor_ classifier will take a *test image*, compare it to every single one of the *training images*, and predict the label of the closest training image.

The _k-Nearest Neighbor_ classifier, instead of taking the single closest image in the training set, we will find the top k closest images, and have them vote on the label of the test image. In particular, when k = 1, we recover the Nearest Neighbor classifier. Intuitively, higher values of k have a smoothing effect that makes the classifier more resistant to outliers:

![](https://cs231n.github.io/assets/knn.jpeg)

In this lab, we will be using the implementation of kNN classification from the [sci-kit learn](https://scikit-learn.org/stable/) library. 

### crossvalidation

Generally speaking, we are looking for image classification methods that can generalize to new images. To measure how well kNN classification generalizes to new images, researchers will typically split their dataset into two sub-datasets: a **training dataset** and a **test dataset**. When reporting how well the classification method works, typically only performance on the "held-out" test dataset is given. To estimate how much uncertainty we have in our estimates of generalization, it is common to use not just one split of the data, but multiple splits. 

#### writing a custom function to get splits from dataframe

In [ ]:
def get_splits(df, 
               train_prop=0.8, 
               random_seed=0,
               replace=False,
               group='category',
               identifier='image_id'):

    ## infer how many observations per group and use to 
    num_obs_per_group = int(df.groupby(group).size().mean())
    size = int(train_prop * num_obs_per_group) ## how many obs do include in train split
    replace = False  # with replacement

    ## create splits
    fn = lambda obj: obj.loc[np.random.RandomState(random_seed).choice(obj.index, size, replace),:]    
    train_split = df.groupby(group, as_index=False).apply(fn)
    common = df.merge(train_split,on=[identifier])
    test_split = df[(~df.image_id.isin(common.image_id))]

    ## sanity check, there is no overlap in image_id
    assert len(np.intersect1d(train_split[identifier],test_split[identifier]))==0
    
    return train_split.reset_index(drop=True), test_split
        

In [ ]:
## apply get_splits function to get splits
train, test = get_splits(V, train_prop=0.75, random_seed=0)

## define training data for kNN classifier
Xtrain = train[np.arange(num_vgg_feats)]
ytrain = train['category'].values

## define test data for kNN classifier
Xtest = test[np.arange(num_vgg_feats)]
ytest = test['category'].values

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

## create an instance of the kNN classifier
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(Xtrain, ytrain)

In [ ]:
## how well did we do at classifying images in the test set?
print(clf.score(Xtest, ytest))

#### k-fold crossvalidation
A common way to perform crossvalidation is known as **k-fold crossvalidation** (no relation to the k in k nearest neighbors). In k-fold cross-validation, the original sample is randomly partitioned into k equal sized subsamples. Of the k subsamples, a single subsample is retained as the validation data for testing the model, and the remaining k − 1 subsamples are used as training data. The cross-validation process is then repeated k times, with each of the k subsamples used exactly once as the validation data. The k results can then be averaged to produce a single estimation. The advantage of this method over repeated random sub-sampling (see below) is that all observations are used for both training and validation, and each observation is used for validation exactly once. 

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=4, random_state=1, shuffle=True)

## entire dataset
X = np.array(V[np.arange(num_vgg_feats)])
y = V['category'].values

## for each split get train/test indices
counter = 0
acc = []
for train_index, test_index in skf.split(X, y):
    Xtrain, Xtest = X[train_index], X[test_index]
    ytrain, ytest = y[train_index], y[test_index]
        
    ## create an instance of the kNN classifier
    clf = KNeighborsClassifier(n_neighbors=3)
    clf.fit(Xtrain, ytrain)

    ## accuracy on test split
    score = clf.score(Xtest, ytest)
    acc.append(score)
    
    ## how well did we do at classifying images in the test set?
    print('Accuracy on fold {} = {}'.format(counter+1, np.round(score,3)))
    counter+=1
print('Mean accuracy = {}'.format(np.round(np.mean(acc),3)))

#### confusion matrix
Each row of the matrix represents the instances in a predicted class while each column represents the instances in an actual class (or vice versa). The name stems from the fact that it makes it easy to see if the system is confusing two classes (i.e. commonly mislabeling one as another).

In [ ]:
## plot confusion matrix
from sklearn.metrics import plot_confusion_matrix        
disp = plot_confusion_matrix(clf, Xtest, ytest,
                             display_labels=np.unique(ytest),
                             cmap=plt.cm.Blues,
                             normalize='true',
                             include_values=False)

### support vector machine classification

Although relatively straightforward to understand, kNN has a number of disadvantages:

- The classifier must remember all of the training data and store it for future comparisons with the test data. This is space inefficient because datasets may easily be gigabytes in size.
- Classifying a test image is expensive since it requires a comparison to all training images.

Next, we'll explore a more powerful approach to image classification that is used much more widely. The approach will have two major components: a **score function** that maps the raw data to class scores, and a **loss function** that quantifies the agreement between the predicted scores and the ground truth labels. We will then cast this as an optimization problem in which we will minimize the loss function with respect to the parameters of the score function.

The first component of this approach is to define the score function that maps the pixel values of an image to confidence scores for each class. We will develop the approach with a concrete example. As before, let’s assume a training dataset of images xi∈RD, each associated with a label yi. Here i=1…N and yi∈1…K. That is, we have N examples (each with a dimensionality D) and K distinct categories. 

**Linear classifier.** In this module we will start out with arguably the simplest possible function, a linear mapping:

$$f(x_i,W,b)=Wx_i+b$$

In the above equation, $x_i$ represents the image feature vector flattened out to a single column vector of shape [D x 1]. The matrix $W$ (of size $[K x D]$), and the vector $b$ (of size $[K x 1]$) are the parameters of the function. In `photodraw32`, $x_i$ either represents all pixels in the i-th image flattened into a single $[12288 x 1]$ column OR the 4096-dimensional feature vector extracted by VGG.

$W$ is $[10 x D]$ and $b$ is $[32 x 1]$, so D numbers come into the function (the elements of the feature vector) and 32 numbers come out (the class scores). The parameters in W are often called the weights, and $b$ is called the bias vector because it influences the output scores, but without interacting with the actual data $x_i$. However, you will often hear people use the terms weights and parameters interchangeably.

There are a few things to note:

- First, note that the single matrix multiplication Wxi is effectively evaluating 10 separate classifiers in parallel (one for each class), where each classifier is a row of W.
- Notice also that we think of the input data ($x_i$,$y_i$) as given and fixed, but we have control over the setting of the parameters W,b. Our goal will be to set these in such way that the computed scores match the ground truth labels across the whole training set. We will go into much more detail about how this is done, but intuitively we wish that the correct class has a score that is higher than the scores of incorrect classes.
- An advantage of this approach is that the training data is used to learn the parameters W,b, but once the learning is complete we can discard the entire training set and only keep the learned parameters. That is because a new test image can be simply forwarded through the function and classified based on the computed scores.
- Lastly, note that classifying the test image involves a single matrix multiplication and addition, which is significantly faster than comparing a test image to all training images.

![](https://cs231n.github.io/assets/imagemap.jpg)

An example of mapping an image to class scores. For the sake of visualization, we assume the image only has 4 pixels (4 monochrome pixels, we are not considering color channels in this example for brevity), and that we have 3 classes (red (cat), green (dog), blue (ship) class). (Clarification: in particular, the colors here simply indicate 3 classes and are not related to the RGB channels.) We stretch the image pixels into a column and perform matrix multiplication to get the scores for each class. Note that this particular set of weights W is not good at all: the weights assign our cat image a very low cat score. In particular, this set of weights seems convinced that it's looking at a dog.

**Analogy of images as high-dimensional points.** We can interpret each feature vector as a single point in a high-dimensional feature space (e.g. each image in photodraw32 is a point in 4096-dimensional space). Analogously, the entire dataset is a (labeled) set of points.

Since we defined the score of each class as a weighted sum of all image pixels, each class score is a linear function over this space. We cannot visualize 4096-dimensional spaces, but if we imagine squashing all those dimensions into only two dimensions, then we can try to visualize what the classifier might be doing:

![](https://cs231n.github.io/assets/pixelspace.jpeg)

Cartoon representation of the image space, where each image is a single point, and three classifiers are visualized. Using the example of the car classifier (in red), the red line shows all points in the space that get a score of zero for the car class. The red arrow shows the direction of increase, so all points to the right of the red line have positive (and linearly increasing) scores, and all points to the left have a negative (and linearly decreasing) scores.

As we saw above, every row of W is a classifier for one of the classes. The geometric interpretation of these numbers is that as we change one of the rows of W, the corresponding line in the pixel space will rotate in different directions. The biases b, on the other hand, allow our classifiers to translate the lines. In particular, note that without the bias terms, plugging in xi=0 would always give score of zero regardless of the weights, so all lines would be forced to cross the origin.

**Interpretation of linear classifiers as template matching.** Another interpretation for the weights W is that each row of W corresponds to a template (or sometimes also called a prototype) for one of the classes. The score of each class for an image is then obtained by comparing each template with the image using an inner product (or dot product) one by one to find the one that “fits” best. With this terminology, the linear classifier is doing template matching, where the templates are learned. Another way to think of it is that we are still effectively doing Nearest Neighbor, but instead of having thousands of training images we are only using a single image per class (although we will learn it, and it does not necessarily have to be one of the images in the training set), and we use the (negative) inner product as the distance instead of the L1 or L2 distance.

![](https://cs231n.github.io/assets/templates.jpg)

#### Loss function
In the previous section we defined a function from the pixel values to class scores, which was parameterized by a set of weights W. Moreover, we saw that we don’t have control over the data $(x_i,y_i)$ (it is fixed and given), but we do have control over these weights and we want to set them so that the predicted class scores are consistent with the ground truth labels in the training data.

For example, going back to the example image of a cat and its scores for the classes “cat”, “dog” and “ship”, we saw that the particular set of weights in that example was not very good at all: We fed in the pixels that depict a cat but the cat score came out very low (-96.8) compared to the other classes (dog score 437.9 and ship score 61.95). We are going to measure our unhappiness with outcomes such as this one with a loss function (or sometimes also referred to as the cost function or the objective). Intuitively, the loss will be high if we’re doing a poor job of classifying the training data, and it will be low if we’re doing well.


##### Multiclass Support Vector Machine loss
There are several ways to define the details of the loss function. As a first example we will first develop a commonly used loss called the Multiclass Support Vector Machine (SVM) loss. The SVM loss is set up so that the SVM “wants” the correct class for each image to a have a score higher than the incorrect classes by some fixed margin Δ. Notice that it’s sometimes helpful to anthropomorphise the loss functions as we did above: The SVM “wants” a certain outcome in the sense that the outcome would yield a lower loss (which is good).

Let’s now get more precise. Recall that for the i-th example we are given the pixels of image xi and the label yi that specifies the index of the correct class. The score function takes the pixels and computes the vector $f(x_i,W)$ of class scores, which we will abbreviate to s (short for scores). For example, the score for the j-th class is the j-th element: $s_j=f(x_i,W)_j$. The Multiclass SVM loss for the i-th example is then formalized as follows:

$$L_i = \sum_{j\neq y_i} \max(0, s_j - s_{y_i} + \Delta) $$


The Multiclass Support Vector Machine "wants" the score of the correct class to be higher than all other scores by at least a margin of delta. If any class has a score inside the red region (or higher), then there will be accumulated loss. Otherwise the loss will be zero. Our objective will be to find the weights that will simultaneously satisfy this constraint for all examples in the training data and give a total loss that is as low as possible.

![](https://cs231n.github.io/assets/margin.jpg)

In [ ]:
## import SVC (support vector classification)
from sklearn.svm import LinearSVC

## apply stratified k-fold crossvalidation, as above, and measure accuracy of your SVM classifier using LinearSVC 

## INSERT YOUR CODE HERE ##


In [ ]:
## plot confusion matrix

## INSERT YOUR CODE HERE ##


### logistic regression ("softmax") classification

It turns out that the SVM is one of two commonly seen classifiers. The other popular choice is the Logistic Regression classifier, which has a different loss function. If you’ve heard of the binary Logistic Regression classifier before, the Softmax classifier is its generalization to multiple classes. Unlike the SVM which treats the outputs $f(x_i,W)$ as (uncalibrated and possibly difficult to interpret) scores for each class, the Softmax classifier gives a slightly more intuitive output (normalized class probabilities) and also has a probabilistic interpretation that we will describe shortly. In the Softmax classifier, the function mapping $f(x_i;W)=Wx_i$ stays unchanged, but we now interpret these scores as the unnormalized log probabilities for each class and replace the hinge loss with a cross-entropy loss that has the form:

$$L_i = -\log\left(\frac{e^{f_{y_i}}}{ \sum_j e^{f_j} }\right) \hspace{0.5in} \text{or equivalently} \hspace{0.5in} L_i = -f_{y_i} + \log\sum_j e^{f_j}$$

In [ ]:
## import SVC (support vector classification)
from sklearn.linear_model import LogisticRegression

## apply stratified k-fold crossvalidation, as above, and measure accuracy of your 
## logistic-regression classifier using LogisticRegression

## INSERT YOUR CODE HERE ##


In [ ]:
## plot confusion matrix


## INSERT YOUR CODE HERE ##